# AirBnB

###  Analytical Exercises using `airbnb` data sets

> Practice problems from real companies; data and questions' source [Stratascratch](https://education.stratascratch.com)

In [1]:
import pandas as pd

In [2]:
%load_ext version_information
%version_information pandas

Software versions
Python 3.7.5 64bit [MSC v.1916 64 bit (AMD64)]
IPython 7.13.0
OS Windows 10 10.0.18362 SP0
pandas 0.25.3
Wed Apr 08 14:58:08 2020 W. Europe Daylight Time

* Find out the searches made by the people who searches for apartments where they will be the sole person staying.



``` python
# python
airbnb.loc[(airbnb.accommodates==1)&(airbnb.beds==1), 'id']
```

``` sql
--sql
select count(id) from airbnb_search_details
where accommodates=1 and beds=1;
```

* Find 50 searches for apartments in New York City which are in the Harlem neighborhood.

``` python
# python
airbnb.loc[(airbnb.city=='NYC')&(airbnb.neighbourhood=='Harlem'), :].head(50)
```

``` sql
-- sql
select * from airbnb_search_details
where city = 'NYC' and neighbourhood = 'Harlem'
limit 50;
```

In [3]:
# Load table airbnb_search_details

airbnb = pd.read_csv('./data/airbnb_search_details.csv')

* Find all searches where the number of bedrooms is equal to the number of bathrooms.

``` python
# python
airbnb[airbnb.bedrooms == airbnb.bathrooms]
```

``` sql
--sql
select * from airbnb_search_details
where bathrooms = bedrooms;
```

* Find Los Angeles neighborhoods for which searches exist.

``` python
# python
airbnb.loc[airbnb.city == 'LA', 'neighbourhood'].sort_values().unique()
```

``` sql
--sql
select distinct neighbourhood from airbnb_search_details
where city='LA'
order by neighbourhood;
```

* Make a pivot table which shows the number of searches per city and room type. Rows are different cities while columns are different room types.

``` python
# python
pd.crosstab(index = airbnb.city, columns = airbnb.room_type)
```

``` sql
-- sql
select distinct room_type, count(*) as quantity from airbnb_search_details
group by room_type;

select
       count(case when room_type = 'Entire home/apt' then room_type end) as entire_property,
       count(case when room_type = 'Private room' then room_type end) as privat_room,
       count(case when room_type = 'Shared room' then room_type end) as shared_room
from airbnb_search_details;

select city, room_type, count(*) as quantity from airbnb_search_details
group by city, room_type;

select city,
       count(case when room_type = 'Entire home/apt' then room_type end) as entire_property,
       count(case when room_type = 'Private room' then room_type end) as privat_room,
       count(case when room_type = 'Shared room' then room_type end) as shared_room
from airbnb_search_details
group by city;
```

* Find all searches where the `host_response_rate` column is missing data.

``` python
# python
airbnb[airbnb.host_response_rate.isna()]
```

``` sql
--sql
select distinct host_response_rate from airbnb_search_details;

select * from airbnb_search_details
where host_response_rate = '';
```

* Find all searches for San Francisco with flexible cancellation policies and which have a review score value. Ordered by the review score from highest to lowest.

``` python
# python
f = (airbnb.city == 'SF') & (airbnb.cancellation_policy == 'flexible')
airbnb[f].sort_values(by=['review_scores_rating', 'id'], ascending=False, axis=0)
```

``` sql
--sql
select * from airbnb_search_details
where city = 'SF' and cancellation_policy = 'flexible'
order by review_scores_rating desc, id desc;
```

* Find all houses and villas which have Internet access but no wireless Internet access.

``` python
# python
f = airbnb.amenities.apply(lambda x: 'Wireless Internet' not in x and 'Internet' in x) 
h = (airbnb.property_type == "House") | (airbnb.property_type == "Villa") 
airbnb[f&h]
```

``` sql
-- sql
select * from airbnb_search_details
where (amenities not like '%Wireless Internet%') and 
    (amenities like '%Internet%') and
    property_type in ('Villa','House');
```

* Find the price of the cheapest property for every city.

``` python
# python
airbnb.groupby('city')['log_price'].min()
```

```sql
--sql
select city, min(log_price) from airbnb_search_details
group by city;
```

* Find all neighborhoods present in this dataset.

``` python
#python
airbnb.neighbourhood.unique()
```

```sql
--sql
select distinct neighbourhood from airbnb_search_details;
```

* Find the search for each city which has the highest number of amenities. Estimate the number of amenities as the number of characters in the `amenities` column.

``` python
# python
airbnb['amenities_len'] = airbnb.amenities.apply(len)
airbnb.groupby(airbnb.city)['amenities_len'].max().sort_values(ascending=False)
```

```sql
--sql
select city, max(amenities_number) from (
select city, length(amenities) as amenities_number from airbnb_search_details)
group by city
order by 2 desc;
```

* Fix the `host_since` column using string processing to be a valid DATE.

``` python
# python
airbnb.loc[:, 'host_since'] = pd.to_datetime(airbnb.host_since)
```

*  Find the price of the most expensive beach properties for each city.

```python
# python
f = airbnb.description.str.lower().apply(lambda x: 'beach' in x)
airbnb[f].groupby('city')['log_price'].max()
```

``` sql
-- sql
SELECT
    city,
    MAX(log_price)
FROM airbnb_search_details
WHERE description LIKE '%beach%'
GROUP BY city;
```

* Find the average number of beds in neighborhoods in which no property has less than 3 beds.

``` python
# python
# find min number of beds per neighbourhood
df = airbnb.groupby(['neighbourhood'])['beds'].min()

# find neighbourhoods in which no property has less yhan 3 beds
nbrhd = df[df>3].index

# filter for neighbourhoods
f = airbnb.neighbourhood.apply(lambda x: x in nbrhd)

# choose properties for selected neighbourhood
airbnb[f].groupby(['neighbourhood'])['beds'].mean()
```

``` sql
--sql
select neighbourhood, avg(beds) from airbnb_search_details
group by neighbourhood
having min(beds) > 3;
```

* It is time for your vacation. You need to chose which city to visit. You don't have much money so you decide to stay in a shared room, but you want to share the room with as little people as possible. You devise a score which tells you the average number of persons accommodated by the shared room over the average number of beds available for each city and order the choice of cities by that score. 

``` python
# python
df = airbnb[airbnb.room_type == 'Shared room'].groupby(['city'])[['accommodates', 'beds']].mean()

(df.accommodates/df.beds).sort_values(ascending=True)
```

``` sql
-- sql
select city, avg(accommodates)/AVG(beds) from airbnb_search_details
where room_type='Shared room'
group by city
order by crowdness_ratio asc;
```

* Find all neighborhoods where there are properties which have no cleaning fees and have parking space.

``` python
# python
f = (airbnb.amenities.str.contains('parking', case=False, regex=False))&
(airbnb.cleaning_fee == False)

airbnb.loc[f, 'neighbourhood'].unique()
```

``` sql
--sql
select distinct neighbourhood from airbnb_search_details
where (amenities like '%parking%') and (cleaning_fee = 'FALSE');
```

* How many hosts are verified by AirBnB staff? How many aren't?

``` python
# python
airbnb.host_identity_verified.value_counts(dropna=False)
```

``` sql
--sql
select distinct host_identity_verified from airbnb_search_details;

select 
    count(case when host_identity_verified = 't' then host_identity_verified end) as verified,
    count(case when host_identity_verified = 'f' then host_identity_verified end) as not_verified,
    count(case when host_identity_verified = '' then host_identity_verified end) as no_information
from airbnb_search_details;

select host_identity_verified, count(host_identity_verified) from airbnb_search_details
group by host_identity_verified; 
```

In [4]:
# Load more tables: airbnb_searches, airbnb_contacts, airbnb_reviews, airbnb_guests

searches = pd.read_csv('./data/airbnb_searches.csv')
contacts = pd.read_csv('./data/airbnb_contacts.csv')
reviews = pd.read_csv('./data/airbnb_reviews.csv')
guests = pd.read_csv('./data/airbnb_guests.csv')
hosts = pd.read_csv('./data/airbnb_hosts.csv')
apartments = pd.read_csv('./data/airbnb_apartments.csv')

* Find the first 5 entries by joining search details and contacts datasets. Tables: `airbnb_contacts`, `airbnb_searches`.

``` python
# python
searches.merge(contacts, how='left', 
                         left_on=['id_user','ds_checkin','ds_checkout'], 
                         right_on=['id_guest','ds_checkin','ds_checkout']).iloc[0:4, :]
```

``` sql
-- sql
select * from airbnb_searches as asd left join airbnb_contacts as ac
on asd.id_user = ac.id_guest and asd.ds_checkin=ac.ds_checkin and asd.ds_checkout=ac.ds_checkout
limit 5;
```

* -- Which gender is more generous in giving positive reviews when considering only guest reviewers? Tables: `airbnb_reviews`, `airbnb_guests`

```python
#python
df = reviews.merge(guests, left_on='from_user', right_on='guest_id', how='left')
df=df[df.from_type=='guest']
df.loc[:, 'review'] = df['review_score'].apply(lambda x: 'positive' if x>5 else 'negative')
df.groupby(['gender', 'review'])['review_score'].count()
```

```sql
--sql
select gender,  
case 
    when review_score > 5 then 'positive'
    when review_score <= 5 then 'negative'
end as review,
count(review_score) from (
    select * from
    airbnb_reviews as r left join airbnb_guests as g
    on r.from_user = g.guest_id
    where r.from_type='guest')
group by gender, review;
```

* Each guest reviews multiple hosts. What is the top rated hosts' nationality for each guest reviewer? Tables: `airbnb_reviews`, `airbnb_hosts`.

```python
#python
# subsettig guest reviews
df = reviews[reviews.to_type=='host'].merge(hosts, left_on='to_user', right_on='host_id', how='left')

# top rated hosts
top_hosts = df[df.review_score==df.review_score.max()]

# top rated hosts' nationality for each reviewer
top_hosts.groupby(['from_user', 'review_score'])['nationality'].unique().apply(lambda x: ', '.join(x))
```

```sql
--sql
select from_user, nationality from (
    select * from
    airbnb_reviews as r left join airbnb_hosts as h
    on r.to_user=host_id
    )
where to_type='host' and review_score = 10
group by from_user, nationality;
```

* How many hosts have apartments in countries of which they are not citizens? Tables: `airbnb_hosts`, `airbnb_apartments`

```python
#python
df = apartments.merge(hosts, on='host_id')
(df.nationality != df.country).sum()
```

```sql
--sql
select count(*) as hosts from (
    select * from airbnb.airbnb_apartments as a inner join airbnb_hosts as h
    on a.host_id=h.host_id
    )
where country <> nationality;
```


* Each host reviews multiple guests. What is the average age of guests reviewed by each host?
Tables: `airbnb_reviews`, `airbnb_guests`.

```python
#python
df=reviews.merge(guests, left_on='to_user', right_on='guest_id', how='inner')
df[df.from_type=='host'].groupby('from_user')['age'].mean()
```

```sql
--sql
select from_user, avg(age)as average_age from (
    select * from 
        airbnb_reviews as r join airbnb_guests as g
        on r.to_user=g.guest_id
   where from_type='host'
)

group by from_user;
```

* Make a pivot table that shows the total number of searches for each room type based on the city. Output should have 4 columns that include the name of the city, apartment count, private room count, and shared room count. Table: `airbnb_search_details`

```python
#python
pd.pivot_table(airbnb, values=['id'], 
               columns=['room_type'], 
               index=['city'], 
               aggfunc='count').reset_index()
```

```sql
--sql
select city,
    count(case when room_type='Entire home/apt' then room_type end) as 'Home/Apt',
    count(case when room_type='Private room' then room_type end) as 'Private room',
    count(case when room_type='Shared room' then room_type end) as 'Shared room'
from airbnb_search_details
group by city;
```

* Find neighborhoods where you can sleep on a real bed in a villa with beach access while paying the minimum price possible. Table: `airbnb_search_details`

```python
# python
f = (airbnb.log_price>0) & \
    (airbnb.bed_type=='Real Bed') & \
    (airbnb.property_type=='Villa') & \
    (airbnb.description.apply(lambda x: 'beach' in x.lower()))

df = airbnb[f]
df.loc[df.log_price==df.log_price.min(), 'neighbourhood']
```

```sql
--sql
select log_price, neighbourhood from airbnb_search_details
where (bed_type = 'Real Bed') and (description like '% beach %') and (property_type='Villa') and log_price in (
    select min(log_price) from airbnb_search_details
    where (log_price > 0) and (bed_type = 'Real Bed') and (description like '% beach %') and (property_type='Villa'));
```

* Find the number of searches made by each user and present the result with their corresponding user id. Table `airbnb_searches`.

```
#python
searches.groupby('id_user')['id_user'].count().reset_index()

```

```sql
--sql
select id_user, count(id_user) from  airbnb_searches
group by id_user;
```

* Display the number of times a user performed a search which led to a successful booking and the number of times a user performed a search but did not lead to a booking. The output should have a column named action with values 'does not book' and 'books' as well as a 2nd column named average_searches with the average number of searches per action. Consider that the booking did not happen if the booking date is null. Tables: `airbnb_searches`, `airbnb_contacts`

```python
# python

# join two tables
df=searches.merge(contacts, how='left',
                              left_on=['id_user','ds_checkin','ds_checkout'], 
                              right_on=['id_guest','ds_checkin','ds_checkout'])
# create a new column that is a categorical variable 'action'
df['action'] = ''
df.loc[df.ts_booking_at.isna(), 'action']='does not book'
df.loc[~df.ts_booking_at.isna(), 'action']='books'

df.groupby(['id_user', 'action'])['n_searches'] \
    .agg(average_searches=('n_searches', 'mean')) \
    .reset_index('action')
```

```sql
--sql
select id_user, 
case
    when ts_booking_at is NULL then 'does not book'
    when ts_booking_at is not NULL then 'books'
end as actions, avg(n_searches) as average_searches from(
    select * from airbnb_searches as asd left join airbnb_contacts as ac
    on asd.id_user = ac.id_guest and asd.ds_checkin=ac.ds_checkin and asd.ds_checkout=ac.ds_checkout
    )
group by id_user, actions;
```

* How many unique users have performed a search? Table: `airbnb_searches`.

```python
#python
searches.id_user.unique().shape
```

```sql
--sql
select count(*) as number_of_users from (
    select distinct id_user from airbnb_searches
);
```


* Find how many the number of different property types in the dataset. Table: `airbnb_searches`.

```python
#python
searches.filter_room_types.str.lstrip(to_strip=',').unique().shape[0]
```

```sql
---sql
select count(*) from (
     select distinct room_types_cleaned from (
        select filter_room_types, 
        case 
             when filter_room_types like ',%' then ltrim(filter_room_types,',')
             when filter_room_types not like ',%' then filter_room_types
        end as room_types_cleaned from airbnb_searches
        )
    );
```